In [1]:
import sys
from pathlib import Path
from pprint import pprint

sys.path.append(str(Path.cwd().parent))


from psychonaut.firehose.serde import read_enriched_event
from psychonaut.core.car import read_car
from psychonaut.client import get_ipynb_client_session
from base64 import b64decode

sess = await get_ipynb_client_session()

In [2]:
with open("../psychonaut/firehose/fixtures/donkeyballs.b64") as fp:
    b64 = fp.read()

post = b64decode(b64)

# Disection of the event stream

- The `repo` refers to the repository that the event is associated with
- The `ops` is?
    - The CID of the op and the CID of teh content change?

In [11]:
event = read_enriched_event(post)
event.dict()

{'repo': 'did:plc:o32okshy54r5h2vlrjpz3aln',
 'ops': [{'action': <ActionType.create: 'create'>,
   'path': 'app.bsky.feed.post/3ju35q7husm2p',
   'cid': CID('base58btc', 1, 'dag-cbor', '1220ada48b7c8394d2855f97e9e47ec0ea63d57778e1af14283ea1a0c2d6a86dc1a0')}],
 'commit': CID('base58btc', 1, 'dag-cbor', '122057a8593ed2fc40ec40c792f26ac497262b99f01c6174fa65e11fe672192f215a'),
 'prev': CID('base58btc', 1, 'dag-cbor', '12201635542e07f43898837f2b8f632fcf260401dc10f799a11c0f27c6a4be7dbab5'),
 'blocks': b':\xa2eroots\x81\xd8*X%\x00\x01q\x12 W\xa8Y>\xd2\xfc@\xec@\xc7\x92\xf2j\xc4\x97&+\x99\xf0\x1cat\xfae\xe1\x1f\xe6r\x19/!Zgversion\x01s\x01q\x12 \xad\xa4\x8b|\x83\x94\xd2\x85_\x97\xe9\xe4~\xc0\xeac\xd5wx\xe1\xaf\x14(>\xa1\xa0\xc2\xd6\xa8m\xc1\xa0\xa3dtextkdonkeyballse$typerapp.bsky.feed.posticreatedAtx\x182023-04-23T23:05:15.184Z\xd1\x01\x01q\x12 \xe7&;\xf7\xf4P\x19\xd1\xf5\x06c\xd2\xc6\xbb!}\x80\xa3\xf2\xe5\xbc\xa6Q\xf7Q\xaa\xb1\xffG\xbe]\xd4\xa2ae\x81\xa4akX app.bsky.feed.like/3jt4gr7d26n2hap\

In [6]:
from psychonaut.api.lexicons.com.atproto.repo.describe_repo import DescribeRepoReq


resolved_handle = await DescribeRepoReq(repo=event.repo).do_xrpc(sess)
resolved_handle.dict()

{'handle': 'generativist.xyz',
 'did': 'did:plc:o32okshy54r5h2vlrjpz3aln',
 'didDoc': {'@context': ['https://www.w3.org/ns/did/v1',
   'https://w3id.org/security/suites/secp256k1-2019/v1'],
  'id': 'did:plc:o32okshy54r5h2vlrjpz3aln',
  'alsoKnownAs': ['at://generativist.xyz'],
  'verificationMethod': [{'id': '#atproto',
    'type': 'EcdsaSecp256k1VerificationKey2019',
    'controller': 'did:plc:o32okshy54r5h2vlrjpz3aln',
    'publicKeyMultibase': 'zQYEBzXeuTM9UR3rfvNag6L3RNAs5pQZyYPsomTsgQhsxLdEgCrPTLgFna8yqCnxPpNT7DBk6Ym3dgPKNu86vt9GR'}],
  'service': [{'id': '#atproto_pds',
    'type': 'AtprotoPersonalDataServer',
    'serviceEndpoint': 'https://bsky.social'}]},
 'collections': ['app.bsky.actor.profile',
  'app.bsky.feed.like',
  'app.bsky.feed.post',
  'app.bsky.feed.repost',
  'app.bsky.graph.block',
  'app.bsky.graph.follow'],
 'handleIsCorrect': True}

So it seems like the ops CID corresponds to the first CID in the CAR blocks.

IDK what the root blocks are though. It seems its the final block in this example?

In [15]:
str(event.ops[0].cid)

'zdpuAx7GYAybGShxy9wvkK5eJt6a5G47tz5z5yeFcDqChfYE3'

In [16]:
roots, blocks = read_car(event.blocks)
str(roots[0]), blocks

('zdpuArKcqh4Bfc5ufSWKTSS1jFRYJ47gpuxCEVXeWdMEjDpAM',
 [Block(cid=zdpuAx7GYAybGShxy9wvkK5eJt6a5G47tz5z5yeFcDqChfYE3, data=[79 bytes]),
  Block(cid=zdpuB1ykUW72jPYKQiPgL5TLsXEjQgMh6EuXeX3WUDhdm6JQs, data=[173 bytes]),
  Block(cid=zdpuAwGe6AfzH6duihBcya3Lx8ttpPhkyNY61wAzhhYwFjhp8, data=[47 bytes]),
  Block(cid=zdpuB1Gzf7TGkaXawv3HUwgPyE4qCktVzpQFNp5Bm1SMQZoyY, data=[1330 bytes]),
  Block(cid=zdpuAy1va7n3FKYGrBhnbCVSRfGM8k1MQH6ywN5XzxZpXTz4o, data=[926 bytes]),
  Block(cid=zdpuAnhA6WihK9nkqhBAjqgZig2b3EWNnDcdFmVhoGB1V1ZdP, data=[544 bytes]),
  Block(cid=zdpuApipxw7YXVnxBhd25eNuzoymDzVNLeXMSdbE8U4hEdRxk, data=[1148 bytes]),
  Block(cid=zdpuAs3qVG5yv8pD966Cm7WySfkQCcfnrmK6b4MkM16eBCxTu, data=[93 bytes]),
  Block(cid=zdpuAvsWfvHv5JMQX8aMSVXcUjVay9eUowWk3xMSrYXapZryq, data=[95 bytes]),
  Block(cid=zdpuArKcqh4Bfc5ufSWKTSS1jFRYJ47gpuxCEVXeWdMEjDpAM, data=[210 bytes])])

In [83]:
cids = {str(block.cid) for block in blocks}
decoded_blocks = [block.decoded for block in blocks]
referenced_cids = {
    str(block['l'])
    for block in decoded_blocks
    if 'l' in block
}
referenced_cids_2 = {
    str(entry['t'])
    for block in decoded_blocks
    if 'e' in block
    for entry in block['e']
}
cids & referenced_cids, cids, referenced_cids, cids & referenced_cids_2

({'zdpuB1Gzf7TGkaXawv3HUwgPyE4qCktVzpQFNp5Bm1SMQZoyY'},
 {'zdpuAnhA6WihK9nkqhBAjqgZig2b3EWNnDcdFmVhoGB1V1ZdP',
  'zdpuApipxw7YXVnxBhd25eNuzoymDzVNLeXMSdbE8U4hEdRxk',
  'zdpuArKcqh4Bfc5ufSWKTSS1jFRYJ47gpuxCEVXeWdMEjDpAM',
  'zdpuAs3qVG5yv8pD966Cm7WySfkQCcfnrmK6b4MkM16eBCxTu',
  'zdpuAvsWfvHv5JMQX8aMSVXcUjVay9eUowWk3xMSrYXapZryq',
  'zdpuAwGe6AfzH6duihBcya3Lx8ttpPhkyNY61wAzhhYwFjhp8',
  'zdpuAx7GYAybGShxy9wvkK5eJt6a5G47tz5z5yeFcDqChfYE3',
  'zdpuAy1va7n3FKYGrBhnbCVSRfGM8k1MQH6ywN5XzxZpXTz4o',
  'zdpuB1Gzf7TGkaXawv3HUwgPyE4qCktVzpQFNp5Bm1SMQZoyY',
  'zdpuB1ykUW72jPYKQiPgL5TLsXEjQgMh6EuXeX3WUDhdm6JQs'},
 {'None',
  'zdpuAnsT9XrtvkWdgLHR58TVdRkHPZckx2A9cDyTDfUYPxnbt',
  'zdpuAqwRzw5mnMz7Udjd7WBA7xHQkhEbSmLaKwCNq1NSqGqs2',
  'zdpuAttVR8V6QEaNKvvTUNSrU4ce6qGGCRjQbHjkNjBjnHCsM',
  'zdpuAzGHdKNxz8sWiUGs3e271vRfefPp5Q2wYzeFfRFs156qX',
  'zdpuB1Gzf7TGkaXawv3HUwgPyE4qCktVzpQFNp5Bm1SMQZoyY'},
 {'zdpuAnhA6WihK9nkqhBAjqgZig2b3EWNnDcdFmVhoGB1V1ZdP',
  'zdpuApipxw7YXVnxBhd25eNuzoymDzVNLeXMSdbE8U4hEdRxk

In [54]:
[(str(block.cid), block.decoded) for block in blocks]

[('zdpuAx7GYAybGShxy9wvkK5eJt6a5G47tz5z5yeFcDqChfYE3',
  {'text': 'donkeyballs',
   '$type': 'app.bsky.feed.post',
   'createdAt': '2023-04-23T23:05:15.184Z'}),
 ('zdpuB1ykUW72jPYKQiPgL5TLsXEjQgMh6EuXeX3WUDhdm6JQs',
  {'e': [{'k': b'app.bsky.feed.like/3jt4gr7d26n2h',
     'p': 0,
     't': CID('base58btc', 1, 'dag-cbor', '1220a12f7427ec109cb7dd0972129a8e8904f5b4d8ef75033aad1ff18c82e4c7153d'),
     'v': CID('base58btc', 1, 'dag-cbor', '122014a86fbbed1fe71f542c69dbb684e2b94c28c83d3c0b97141155de51e6517fc7')}],
   'l': CID('base58btc', 1, 'dag-cbor', '1220cdab60836f20c405e627337e8d2e5a180ab0c7fdba9070ffdbf024eb11bc6012')}),
 ('zdpuAwGe6AfzH6duihBcya3Lx8ttpPhkyNY61wAzhhYwFjhp8',
  {'e': [],
   'l': CID('base58btc', 1, 'dag-cbor', '1220dcb5752fd2dda81d78a917a51f16da58f7ab1a10955f078a7ec2a6c0161bcf77')}),
 ('zdpuB1Gzf7TGkaXawv3HUwgPyE4qCktVzpQFNp5Bm1SMQZoyY',
  {'e': [{'k': b'app.bsky.feed.like/3jt7ahzizs22a',
     'p': 0,
     't': CID('base58btc', 1, 'dag-cbor', '12202e7905857fa49f1e7594a6f

In [76]:
[block.decoded for block in blocks]

[{'text': 'donkeyballs',
  '$type': 'app.bsky.feed.post',
  'createdAt': '2023-04-23T23:05:15.184Z'},
 {'e': [{'k': b'app.bsky.feed.like/3jt4gr7d26n2h',
    'p': 0,
    't': CID('base58btc', 1, 'dag-cbor', '1220a12f7427ec109cb7dd0972129a8e8904f5b4d8ef75033aad1ff18c82e4c7153d'),
    'v': CID('base58btc', 1, 'dag-cbor', '122014a86fbbed1fe71f542c69dbb684e2b94c28c83d3c0b97141155de51e6517fc7')}],
  'l': CID('base58btc', 1, 'dag-cbor', '1220cdab60836f20c405e627337e8d2e5a180ab0c7fdba9070ffdbf024eb11bc6012')},
 {'e': [],
  'l': CID('base58btc', 1, 'dag-cbor', '1220dcb5752fd2dda81d78a917a51f16da58f7ab1a10955f078a7ec2a6c0161bcf77')},
 {'e': [{'k': b'app.bsky.feed.like/3jt7ahzizs22a',
    'p': 0,
    't': CID('base58btc', 1, 'dag-cbor', '12202e7905857fa49f1e7594a6fb4ccc8c34a7ffa65eb04f31d994e0957ec08e41e7'),
    'v': CID('base58btc', 1, 'dag-cbor', '1220949607289cf1c2b45f4a6dcd72fd54e8bc178360d42786d246a5f9efddf074a8')},
   {'k': b'pgweovu2i',
    'p': 23,
    't': CID('base58btc', 1, 'dag-cbor',

This is the CID of the content in value,

In [22]:
from multiformats import CID
from psychonaut.api.lexicons.com.atproto.repo.get_record import GetRecordReq


collection, rkey = str(event.ops[0].path).split("/")
req_resp = (await GetRecordReq(
    repo=event.repo,
    collection=collection,
    rkey=rkey,
).do_xrpc(sess))
print(CID.decode(req_resp.cid))
req_resp.dict()

bafyreifnusfxza4u2kcv7f7j4r7mb2td2v3xrynpcqud5inayllkq3obua


{'uri': 'at://did:plc:o32okshy54r5h2vlrjpz3aln/app.bsky.feed.post/3ju35q7husm2p',
 'cid': 'bafyreifnusfxza4u2kcv7f7j4r7mb2td2v3xrynpcqud5inayllkq3obua',
 'value': {'text': 'donkeyballs',
  '$type': 'app.bsky.feed.post',
  'createdAt': '2023-04-23T23:05:15.184Z'}}

In [62]:
CID.decode(req_resp.cid), blocks[0].cid # Not sure why they have different CID defaults

(CID('base32', 1, 'dag-cbor', '1220ada48b7c8394d2855f97e9e47ec0ea63d57778e1af14283ea1a0c2d6a86dc1a0'),
 CID('base58btc', 1, 'dag-cbor', '1220ada48b7c8394d2855f97e9e47ec0ea63d57778e1af14283ea1a0c2d6a86dc1a0'))

In [74]:
CID.decode(req_resp.cid).encode("base58btc"), str(blocks[0].cid)  # They decode the same

('zdpuAx7GYAybGShxy9wvkK5eJt6a5G47tz5z5yeFcDqChfYE3',
 'zdpuAx7GYAybGShxy9wvkK5eJt6a5G47tz5z5yeFcDqChfYE3')

In [27]:
str(event.commit)

'zdpuArKcqh4Bfc5ufSWKTSS1jFRYJ47gpuxCEVXeWdMEjDpAM'

In [32]:
event.commit

CID('base58btc', 1, 'dag-cbor', '122057a8593ed2fc40ec40c792f26ac497262b99f01c6174fa65e11fe672192f215a')

In [51]:
import dag_cbor

(
    CID.decode(req_resp.cid),
    event.commit.hashfun.digest(dag_cbor.encode(req_resp.value)).hex(),
)

(CID('base32', 1, 'dag-cbor', '1220ada48b7c8394d2855f97e9e47ec0ea63d57778e1af14283ea1a0c2d6a86dc1a0'),
 '1220ada48b7c8394d2855f97e9e47ec0ea63d57778e1af14283ea1a0c2d6a86dc1a0')

CID('base32', 1, 'dag-cbor', '1220ada48b7c8394d2855f97e9e47ec0ea63d57778e1af14283ea1a0c2d6a86dc1a0')

In [40]:
(await GetRecordReq(
    repo=repo,
    collection="app.bsky.feed.post",
    rkey="3jt76otcoqk2a",
).do_xrpc(sess)).dict()

{'uri': 'at://did:plc:o32okshy54r5h2vlrjpz3aln/app.bsky.feed.post/3jt76otcoqk2a',
 'cid': 'bafyreig37om67if6vpcyssn4gbioibcqwzbtrj53eetaw2dlmyd4sgewd4',
 'value': {'text': 'yes for a nice long nap',
  '$type': 'app.bsky.feed.post',
  'reply': {'root': {'cid': 'bafyreibuht7vakzsxyglaxvo5ntxguyy6s22d7x27b6hnb5oztzqtigcmy',
    'uri': 'at://did:plc:o32okshy54r5h2vlrjpz3aln/app.bsky.feed.post/3jt76mavus22z'},
   'parent': {'cid': 'bafyreiedtpfltjxxrzeo4r6uk7ols3e6umn6gsx6eearexyq3je3rocbfi',
    'uri': 'at://did:plc:4rnej33af5ly4yjgpuaeupd3/app.bsky.feed.post/3jt76ndfzsd2q'}},
  'createdAt': '2023-04-12T20:07:49.991Z'}}

In [34]:
from multiformats import varint

varint

<module 'multiformats.varint' from '/home/generativist/.cache/pypoetry/virtualenvs/psychonaut-hgjsIY8y-py3.10/lib/python3.10/site-packages/multiformats/varint/__init__.py'>